## Helper functions for sentiment analysis

In [ ]:
import re
from openai import OpenAI
client = OpenAI(api_key = "******")#hiding the prompt key for security reasons

def get_openai_api(inp, temp=0,topP=1):

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": inp}
        ],
        temperature = temp,
        top_p = topP,
        stream=True
    )
    return completion.choices[0].message.content

In [ ]:
#processes prompt
def get_answer_llm(user_prompt):

    sys_prompt="You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
    prompt = user_prompt
    try:
        return get_openai_api(prompt)
    except:
        print("RATE LIMITTT")
        return "SOME PROBLEM IN LLM GPT SEE"

In [ ]:
#this is a regex function that extracts the scores from the output of the LLM
def extract_scores(sent):
#   print(sent)
  sent=sent.lower()
  ans={}
  try:
    pattern1 = r'<trump_favour>(.*?)</trump_favour>'
    matches1 = re.findall(pattern1, sent)
    ans['trump_favour']=float(matches1[0])

    pattern2 = r'<biden_favour>(.*?)</biden_favour>'
    matches2 = re.findall(pattern2, sent)
    ans['biden_favour']=float(matches2[0])

    pattern3 = r'<dem_favour>(.*?)</dem_favour>'
    matches3 = re.findall(pattern3, sent)
    ans['dem_favour']=float(matches3[0])


    pattern4 = r'<repub_favour>(.*?)</repub_favour>'
    matches4 = re.findall(pattern4, sent)
    ans['repub_favour']=float(matches4[0])

#     pattern5 = r'<other>(.*?)</other>'
#     matches5 = re.findall(pattern5, sent)
#     ans['other']=float(matches5[0])
  except:
    return {}

  return ans

# For reddit

In [ ]:
import random
import csv
from tqdm import tqdm
import time

# Specify the CSV file name
csv_file = 'redditrepub_output_senti_sampled_sleep_NOVDEC1000.csv'

# Extract column headers from the first dictionary in the list
fieldnames = ['trump_favour', 'biden_favour', 'dem_favour', 'repub_favour', 'text']
def read_lines(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Read all lines from the file
        all_lines = file.readlines()

    return all_lines

file_path = 'Republicans.txt'
sampled_lines = read_lines(file_path)


with open(csv_file, 'w', newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    for idx,data in enumerate(sampled_lines):
        prompt=prompt="given this input sentence:"+data+"Give a score between 0-1 for the following 4 attributes:"+"\n"+"1. Favoribility of Trump \n 2. Favouribility of Biden \n 3. Favouribility of Democrats \n 4. Favouribility of Republicans"+"Answer these in the following form between the following tags:"+"1. <trump favour> and  </trump_favour> \n 2. <biden_favour> and  </biden_favour> \n 3.<dem_favour> and  </dem_favour> \n 4. <repub_favour> and  </repub_favour> \n"+"Treat this task as a task to rate the input sentence on the basis of political favoribility. Just give the scores between the tags as output, nothing else as output.But necessarily give a score to all 4 tags."
        time.sleep(5)#to not hit rate limit
        ans=extract_scores(get_answer_llm(prompt))
        if ans=={}:
            continue
        ans['text']=data.strip()
        print(ans)
        writer.writerow(ans)
        if idx%100==0:
            print("Completed",idx)



# For news api

In [ ]:
import random
import csv
from tqdm import tqdm
import time
import pandas as pd

# Specify the CSV file name
csv_file = 'WITHDATE_newsapi_output_senti_400.csv'

# Extract column headers from the first dictionary in the list
fieldnames = ['trump_favour', 'biden_favour', 'dem_favour', 'repub_favour', 'text',"publishedAt"]

file_path = 'output_newsapi_improved_withdate.csv'
df=pd.read_csv(file_path)
df['combined'] =df['publishedAt'].astype(str)+'D|D'+ df['title'].astype(str) +". "+ df['description'].astype(str)

sampled_lines=list(df['combined'])

# for i in sampled_lines:
#     print(i)
#     print('**********')

with open(csv_file, 'w', newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    for idx,data in enumerate(sampled_lines):#i have kept tqdm at all linesto not hit rate limit

            date,data=data.split("D|D")
            prompt=prompt="given this input sentence:"+data+"Give a score between 0-1 for the following 4 attributes:"+"\n"+"1. Favoribility of Trump \n 2. Favouribility of Biden \n 3. Favouribility of Democrats \n 4. Favouribility of Republicans"+"Answer these in the following form between the following tags:"+"1. <trump favour> and  </trump_favour> \n 2. <biden_favour> and  </biden_favour> \n 3.<dem_favour> and  </dem_favour> \n 4. <repub_favour> and  </repub_favour> \n"+"Treat this task as a task to rate the input sentence on the basis of political favoribility. Just give the scores between the tags as output, nothing else as output.But necessarily give a score to all 4 tags."
            time.sleep(5)
            ans=extract_scores(get_answer_llm(prompt))
            if ans=={}:
                continue
            ans['text']=data.strip()
            ans['publishedAt']=date.strip()
            print(ans)
            writer.writerow(ans)
            if idx%100==0:
                print("Completed",idx)




{'trump_favour': 0.0, 'biden_favour': 0.0, 'dem_favour': 0.0, 'repub_favour': 1.0, 'text': 'GOP Sen. Johnson: \'Obvious\' Biden is \'corrupt\' and \'compromised\'. "And they were influenced peddling. And the people paying these millions expected favors in return some some kind of public policy coming from the former vice president and maybe even a potential president of the United States. Now they got him now, he\'s comp…', 'publishedAt': '2023-12-08T18:57:10Z'}
Completed 0
{'trump_favour': 0.0, 'biden_favour': 1.0, 'dem_favour': 1.0, 'repub_favour': 0.0, 'text': 'LOOK AT THIS COMMUNITY NOTE AND IT’S EASY TO SEE WHY THEY HATE ELON: https://twitter.com/JoeBiden. LOOK AT THIS COMMUNITY NOTE AND IT’S EASY TO SEE WHY THEY HATE ELON: We’ve created close to 800,000 manufacturing jobs since I’ve taken office. — Joe Biden (@JoeBiden) December 5, 2023 Haha you got fact checked. — Melanie (@Imagecaptured) December 5, 2023', 'publishedAt': '2023-12-07T14:21:47Z'}
{'trump_favour': 0.8, 'biden_favo

# google news

In [ ]:
import random
import csv
from tqdm import tqdm
import time

# Specify the CSV file name
csv_file = 'WITHDATE_googlenews_output_senti_1000.csv'

# Extract column headers from the first dictionary in the list
fieldnames = ['trump_favour', 'biden_favour', 'dem_favour', 'repub_favour', 'text']



file_path = 'cleaned_news.txt'
sampled_lines = read_lines(file_path)


with open(csv_file, 'w', newline='') as csvfile:
    # Create a CSV writer object
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()

    for idx,data in enumerate(sampled_lines):#i have kept tqdm at all linesto not hit rate limit
        prompt="given this input sentence:"+data+"Give a score between 0-1 for the following 4 attributes:"+"\n"+"1. Favoribility of Trump \n 2. Favouribility of Biden \n 3. Favouribility of Democrats \n 4. Favouribility of Republicans"+"Answer these in the following form between the following tags:"+"1. <trump favour> and  </trump_favour> \n 2. <biden_favour> and  </biden_favour> \n 3.<dem_favour> and  </dem_favour> \n 4. <repub_favour> and  </repub_favour> \n"+"Treat this task as a task to rate the input sentence on the basis of political favoribility. Just give the scores between the tags as output, nothing else as output.But necessarily give a score to all 4 tags."
        time.sleep(5)
        ans=extract_scores(get_answer_llm(prompt))
        if ans=={}:
            continue
        ans['text']=data.strip()
        print(ans)
        writer.writerow(ans)
        if idx%100==0:
            print("Completed",idx)



77772
7782
{'trump_favour': 0.2, 'biden_favour': 0.6, 'dem_favour': 0.3, 'repub_favour': 0.4, 'text': 'Date: 11/20/2023 | Source: Carolina Public Press | Title: Muslim North Carolinians say they feel alienated by Democrats'}
Completed 0
{'trump_favour': 0.8, 'biden_favour': 0.2, 'dem_favour': 0.3, 'repub_favour': 0.7, 'text': 'Date: 11/17/2023 | Source: WLRN | Title: Poll: Florida Hispanic voters favor Trump over Biden in 2024 election'}
{'trump_favour': 0.2, 'biden_favour': 0.6, 'dem_favour': 0.5, 'repub_favour': 0.4, 'text': 'Date: 11/07/2023 | Source: Vox.com | Title: Why are Biden polls so bad? Here are 3 leading theories.'}
{'trump_favour': 0.5, 'biden_favour': 0.3, 'dem_favour': 0.4, 'repub_favour': 0.6, 'text': 'Date: 11/07/2023 | Source: NBC News | Title: George Santos’ exit sets up a competitive special election'}
{'trump_favour': 0.5, 'biden_favour': 0.3, 'dem_favour': 0.4, 'repub_favour': 0.6, 'text': 'Date: 11/09/2023 | Source: The Nation | Title: Forget the Polls—Here’s Wh

In [2]:
p="given this input sentence: [DATA] Give a score between 0-1 for the following 4 attributes:"+"\n"+"1. Favoribility of Trump \n 2. Favouribility of Biden \n 3. Favouribility of Democrats \n 4. Favouribility of Republicans"+"Answer these in the following form between the following tags:"+"1. <trump favour> and  </trump_favour> \n 2. <biden_favour> and  </biden_favour> \n 3.<dem_favour> and  </dem_favour> \n 4. <repub_favour> and  </repub_favour> \n"+"Treat this task as a task to rate the input sentence on the basis of political favoribility. Just give the scores between the tags as output, nothing else as output.But necessarily give a score to all 4 tags."
p

'given this input sentence: [DATA] Give a score between 0-1 for the following 4 attributes:\n1. Favoribility of Trump \n 2. Favouribility of Biden \n 3. Favouribility of Democrats \n 4. Favouribility of RepublicansAnswer these in the following form between the following tags:1. <trump favour> and  </trump_favour> \n 2. <biden_favour> and  </biden_favour> \n 3.<dem_favour> and  </dem_favour> \n 4. <repub_favour> and  </repub_favour> \nTreat this task as a task to rate the input sentence on the basis of political favoribility. Just give the scores between the tags as output, nothing else as output.But necessarily give a score to all 4 tags.'